In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
import csv  # For saving hyperparameters, predictions, and metrics
import torch
import torch.nn as nn
import torch.optim as optim

# Load and preprocess data
def load_data(features_file, labels_file, labels_column):
    features_df = pd.read_csv(features_file, index_col=0)
    labels_df = pd.read_csv(labels_file, index_col=0)

    X = features_df
    y = labels_df[labels_column]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y

# Save the best hyperparameters to a CSV file
def save_hyperparameters(model_name, best_params):
    filename = f"{model_name}_best_params.csv"
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Parameter", "Value"])
        # Assuming best_params is a list of dictionaries, one for each fold
        for params in best_params:
            for param, value in params.items():
                writer.writerow([param, value])
    print(f"Best hyperparameters for {model_name} saved to {filename}")

# Save the predictions, true labels, and metrics to a CSV file
def save_predictions_and_metrics(model_name, y_true, y_pred, corr, rmse):
    # Save predictions and true labels
    predictions_filename = f"../Results/{model_name}_predictions.csv"
    with open(predictions_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["True Label", "Predicted Label"])
        writer.writerows(zip(y_true, y_pred))
    
    # Save correlation and RMSE metrics
    metrics_filename = f"../Results/{model_name}_metrics.csv"
    with open(metrics_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Metric", "Value"])
        writer.writerow(["Correlation", corr])
        writer.writerow(["RMSE", rmse])
    
    print(f"Predictions for {model_name} saved to {predictions_filename}")
    print(f"Metrics for {model_name} saved to {metrics_filename}")

# Perform Leave-One-Out Cross-Validation on a single model
def run_loo_cv(model, param_grid, model_name, X, y):
    loo = LeaveOneOut()
    y_true, y_pred, best_params = [], [], []

    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        y_true.append(y_test.values[0])
        
        if param_grid:
            # Add n_jobs=-1 for parallel processing during Grid Search
            grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            pred = best_model.predict(X_test)
            best_params.append(grid_search.best_params_)
        else:
            model.fit(X_train, y_train)
            pred = model.predict(X_test)
        
        y_pred.append(pred[0])
    
    corr, _ = pearsonr(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    
    print(f"{model_name}: Correlation (R) = {corr:.4f}, RMSE = {rmse:.4f}")
    
    # Save hyperparameters, predictions, and metrics
    if param_grid:
        print(f"Best hyperparameters for {model_name}: {best_params}")
        save_hyperparameters(model_name, best_params)
    
    save_predictions_and_metrics(model_name, y_true, y_pred, corr, rmse)

    return best_params

# Generic training and evaluation function
def train_and_evaluate(model_class, X, y, num_epochs=200, learning_rate=0.001):
    # Check if a GPU is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Determine the number of input features
    input_size = X.shape[1]
    
    # Initialize Leave-One-Out cross-validator
    loo = LeaveOneOut()
    y_true, y_pred = [], []

    for train_index, test_index in loo.split(X):
        # Split the data into training and test for this fold
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Convert to PyTorch tensors and move to GPU if available
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
        y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

        # Initialize the model, loss function, and optimizer
        model = model_class(input_size=input_size).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Training loop
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()
            outputs = model(X_train_tensor)
            loss = criterion(outputs, y_train_tensor)
            loss.backward()
            optimizer.step()

            # Print loss every 10 epochs
            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

        # Evaluate on the left-out test sample
        model.eval()
        with torch.no_grad():
            prediction = model(X_test_tensor).cpu().numpy()[0][0]  # Get prediction and move back to CPU

        # Store the true value and predicted value
        y_true.append(y_test.values[0])
        y_pred.append(prediction)

    # Calculate RMSE and Pearson correlation coefficient (R-value)
    test_rmse = mean_squared_error(y_true, y_pred, squared=False)
    corr = pearsonr(y_true, y_pred)

    # Print the metrics
    print(f"\nModel: {model_class.__name__} | Test RMSE: {test_rmse:.4f} | Test R value (correlation): {corr[0]:.4f}")

    # Save predictions and metrics using the existing function
    model_name = model_class.__name__
    save_predictions_and_metrics(model_name, y_true, y_pred, corr[0], test_rmse)

# Choose and run individual models
def run_linear_regression(X, y):
    model_name = 'Linear_Regression'
    model = LinearRegression()
    best_params = run_loo_cv(model, None, model_name, X, y)

def run_ridge(X, y):
    model_name = 'Ridge'
    model = Ridge()
    best_params = run_loo_cv(model, param_grids[model_name], model_name, X, y)

def run_lasso(X, y):
    model_name = 'Lasso'
    model = Lasso()
    best_params = run_loo_cv(model, param_grids[model_name], model_name, X, y)

def run_svr(X, y):
    model_name = 'SVR'
    model = SVR()
    best_params = run_loo_cv(model, param_grids[model_name], model_name, X, y)

def run_random_forest(X, y):
    model_name = 'Random Forest'
    model = RandomForestRegressor(n_jobs=-1)  # Enable parallel processing for RandomForest
    best_params = run_loo_cv(model, param_grids[model_name], model_name, X, y)

def run_xgboost(X, y):
    model_name = 'XGBoost'
    model = XGBRegressor(use_label_encoder=False, eval_metric='rmse', n_jobs=-1)  # Enable parallel processing for XGBoost
    best_params = run_loo_cv(model, param_grids[model_name], model_name, X, y)

# Define parameter grids for models
param_grids = {
    'Ridge': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]},
    'Lasso': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]},
    'SVR': {
        # 'C': [0.1, 1.0, 10.0, 100.0],
        # 'epsilon': [0.001, 0.01, 0.1, 1.0],
        # 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
    },
    'Random Forest': {
        # 'n_estimators': [100, 200],
        # 'max_depth': [None, 10, 20],
        # 'min_samples_split': [2, 5],
        # 'min_samples_leaf': [1, 2]
    },
    'XGBoost': {
        'n_estimators': [100, 200],
        'learning_rate': [0.001, 0.01, 0.1, 0.3],
        'max_depth': [3, 5, 10],
        'subsample': [0.6, 0.8, 1.0]
    }
}

class NeuralNetMLP(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class DeepNeuralNetMLP(nn.Module):
    def __init__(self, input_size):
        super(DeepNeuralNetMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc5(x)
        return x

class CNN(nn.Module):
    def __init__(self, input_size):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.relu = nn.ReLU()
        # Calculate the flattened size after convolution and pooling
        self.flattened_size = self._get_flattened_size(input_size)

        # Define fully connected layers
        self.fc1 = nn.Linear(self.flattened_size, 128)
        self.fc2 = nn.Linear(128, 1)


    def _get_flattened_size(self, input_size):
        # Create a dummy tensor with the same size as the input to calculate the output size after convolutions
        dummy_input = torch.zeros(1, 1, input_size)  # Batch size of 1, 1 channel, input_size as width
        output = self.pool(self.relu(self.conv1(dummy_input)))
        output = self.pool(self.relu(self.conv2(output)))
        flattened_size = output.numel()  # Calculate the number of elements in the output tensor
        return flattened_size

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Main workflow
if __name__ == '__main__':
    features_file = '../Processed Data/rest_101_participants_40_regions.csv'
    labels_file = '../Processed Data/101_participants_40_regions_target_variable.csv'
    prediction_label = 'Aphasia quotient'
    
    X, y = load_data(features_file, labels_file, prediction_label)
    
    # Choose a model to run (uncomment the model you want to run)
    #run_linear_regression(X, y)
    # run_ridge(X, y)
    # run_lasso(X, y)
    #run_svr(X, y)
    #run_random_forest(X, y)
    # run_xgboost(X, y)
    #train_and_evaluate(NeuralNetMLP, X, y)
    #train_and_evaluate(DeepNeuralNetMLP, X, y)
    train_and_evaluate(CNN, X, y)


Using device: cuda
Epoch [10/200], Loss: 1512.8531
Epoch [20/200], Loss: 1060.5099
Epoch [30/200], Loss: 786.2903
Epoch [40/200], Loss: 648.0015
Epoch [50/200], Loss: 612.6757
Epoch [60/200], Loss: 575.7255
Epoch [70/200], Loss: 533.0200
Epoch [80/200], Loss: 482.3949
Epoch [90/200], Loss: 426.4696
Epoch [100/200], Loss: 366.3800
Epoch [110/200], Loss: 305.9165
Epoch [120/200], Loss: 253.4401
Epoch [130/200], Loss: 212.5565
Epoch [140/200], Loss: 176.9109
Epoch [150/200], Loss: 144.1169
Epoch [160/200], Loss: 115.8860
Epoch [170/200], Loss: 92.2720
Epoch [180/200], Loss: 73.1074
Epoch [190/200], Loss: 58.1461
Epoch [200/200], Loss: 46.9550
Epoch [10/200], Loss: 700.6831
Epoch [20/200], Loss: 657.5849
Epoch [30/200], Loss: 684.2012
Epoch [40/200], Loss: 608.9495
Epoch [50/200], Loss: 543.2822
Epoch [60/200], Loss: 488.8678
Epoch [70/200], Loss: 433.9113
Epoch [80/200], Loss: 371.4339
Epoch [90/200], Loss: 308.7489
Epoch [100/200], Loss: 250.1938
Epoch [110/200], Loss: 201.5726
Epoch [12

C:\Users\shark\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [48]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_mean_pool

from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F 
from torch_geometric.loader import DataLoader
from sklearn.model_selection import LeaveOneOut

def load_data_3d(input_folder, labels_file, labels_column, normalization='minmax'):
    labels_df = pd.read_csv(labels_file, index_col=0)
    y = labels_df[labels_column]
    data_dict = {}  
    scaler = MinMaxScaler()  # Create a scaler instance

    for filename in os.listdir(input_folder):
        if filename.endswith('.csv'):
            # Files look like this 'M10011_rest_jhu.csv', extract the number
            parts = filename.split('_')
            if len(parts) > 0:
                participant_id = ''.join(filter(str.isdigit, parts[0]))
                csv_path = os.path.join(input_folder, filename)
                matrix = pd.read_csv(csv_path, index_col=None, header=None).values

                # Normalize the matrix based on the chosen method
                if normalization == 'minmax':
                    matrix = scaler.fit_transform(matrix)  # Min-Max scale between 0 and 1
                elif normalization == 'standard':
                    matrix = (matrix - np.mean(matrix)) / np.std(matrix)  # Standardize to have mean 0, std 1
                
                data_dict[participant_id] = matrix

    return data_dict, y

def convert_dict_to_array(data_dict):
    matrix_list = [data_dict[participant_id] for participant_id in data_dict]
    
    # Convert the list of matrices to a 3D numpy array
    X_array = np.array(matrix_list)
    return X_array

def get_shape_3d(data_dict):
    num_participants = len(data_dict)
    matrix_shape = next(iter(data_dict.values())).shape if num_participants > 0 else (0, 0)
    #print(f"Number of Participants: {num_participants}, Matrix Shape per Participant: {matrix_shape}")
    return  num_participants, matrix_shape[0]

def train_and_evaluate_3d(model_class, X_dict, y, num_epochs=200, learning_rate=0.001):
    # Check if a GPU is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Convert the dictionary to a 3D array
    X_array = convert_dict_to_array(X_dict)

    # Get the shape of each 2D matrix
    num_samples, num_regions, _ = X_array.shape
    
    # Initialize Leave-One-Out cross-validator
    loo = LeaveOneOut()
    y_true, y_pred = [], []

    for train_index, test_index in loo.split(X_array):
        # Split the data into training and test for this fold
        X_train, X_test = X_array[train_index], X_array[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # For CNN, reshape input into (batch_size, 1, num_regions, num_regions)
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1).to(device)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1).to(device)
        
        # For MLP, flatten the 2D matrices into vectors of size (batch_size, num_regions * num_regions)
        if model_class.__name__ in ["NeuralNetMLP", "DeepNeuralNetMLP"]:
            X_train_tensor = X_train_tensor.view(X_train_tensor.size(0), -1)
            X_test_tensor = X_test_tensor.view(X_test_tensor.size(0), -1)
        
        y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
        y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

        # Initialize the model, loss function, and optimizer
        input_size = X_train_tensor.size(1) if model_class.__name__ in ["NeuralNetMLP", "DeepNeuralNetMLP"] else num_regions
        model = model_class(input_size=input_size).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Training loop
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()
            outputs = model(X_train_tensor)
            loss = criterion(outputs, y_train_tensor)
            loss.backward()
            optimizer.step()

            # Print loss every 10 epochs
            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

        # Evaluate on the left-out test sample
        model.eval()
        with torch.no_grad():
            prediction = model(X_test_tensor).cpu().numpy()[0][0]  # Get prediction and move back to CPU

        # Store the true value and predicted value
        y_true.append(y_test.values[0])
        y_pred.append(prediction)

    # Calculate RMSE and Pearson correlation coefficient (R-value)
    test_rmse = mean_squared_error(y_true, y_pred, squared=False)
    corr = pearsonr(y_true, y_pred)

    # Print the metrics
    print(f"\nModel: {model_class.__name__} | Test RMSE: {test_rmse:.4f} | Test R value (correlation): {corr[0]:.4f}")

    # Save predictions and metrics using the existing function
    model_name = model_class.__name__
    save_predictions_and_metrics(model_name, y_true, y_pred, corr[0], test_rmse)

class CNN_2d(nn.Module):
    def __init__(self, input_size):
        super(CNN_2d, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.relu = nn.ReLU()
        
        # Calculate the flattened size after convolution and pooling
        self.flattened_size = self._get_flattened_size(input_size)

        # Define fully connected layers
        self.fc1 = nn.Linear(self.flattened_size, 128)
        self.fc2 = nn.Linear(128, 1)

    def _get_flattened_size(self, input_size):
        # Create a dummy tensor with the same size as the input to calculate the output size after convolutions
        dummy_input = torch.zeros(1, 1, input_size, input_size)  # Batch size of 1, 1 channel, input_size as height and width
        output = self.pool(self.relu(self.conv1(dummy_input)))
        output = self.pool(self.relu(self.conv2(output)))
        flattened_size = output.numel()  # Calculate the number of elements in the output tensor
        return flattened_size

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def convert_to_graph(matrix):
    num_nodes = matrix.shape[0]
    
    # Create an edge index based on non-zero correlations (can be thresholded if needed)
    edge_index = torch.nonzero(torch.tensor(matrix), as_tuple=False).t().contiguous()
    edge_weight = torch.tensor(matrix[edge_index[0], edge_index[1]], dtype=torch.float)

    # Node features can be set as identity matrix or something else relevant
    x = torch.eye(num_nodes, dtype=torch.float)  # Using an identity matrix as node features

    # Create a Data object for PyTorch Geometric
    graph_data = Data(x=x, edge_index=edge_index, edge_attr=edge_weight)
    return graph_data

def prepare_graph_data(data_dict, y):
    """
    Convert the entire dataset into a list of PyTorch Geometric Data objects.
    """
    graph_data_list = []
    for i, (participant_id, matrix) in enumerate(data_dict.items()):
        graph = convert_to_graph(matrix)
        graph.y = torch.tensor([y.iloc[i]], dtype=torch.float)  # Attach the target variable as the label
        graph_data_list.append(graph)
    return graph_data_list

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 16)
        self.fc1 = torch.nn.Linear(16, 1)  # Final fully connected layer for regression

    def forward(self, data):
        # Extract data from the PyTorch Geometric Data object
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr

        # GCN layers with ReLU activation
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.relu(self.conv2(x, edge_index, edge_weight))

        # Global pooling layer to summarize the graph
        x = global_mean_pool(x, data.batch)

        # Final fully connected layer
        x = self.fc1(x)
        return x
    


def train_and_evaluate_gnn(model_class, X_dict, y, num_epochs=200, learning_rate=0.001):
    # Check if a GPU is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Convert the dictionary to a list of graph data objects
    graph_data_list = prepare_graph_data(X_dict, y)

    # Move all graph data objects to the same device
    for graph_data in graph_data_list:
        graph_data.to(device)

    # Initialize Leave-One-Out cross-validator
    loo = LeaveOneOut()
    y_true, y_pred = [], []

    for train_index, test_index in loo.split(graph_data_list):
        # Split the data into training and test for this fold
        train_data = [graph_data_list[i] for i in train_index]
        test_data = graph_data_list[test_index[0]].to(device)

        # Create DataLoader for the training data
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

        # Initialize the model, loss function, and optimizer
        num_node_features = train_data[0].x.shape[1]  # Assuming all graphs have the same number of features
        model = model_class(num_node_features=num_node_features, hidden_channels=32).to(device)
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        # Training loop
        for epoch in range(num_epochs):
            model.train()
            total_loss = 0
            for batch in train_loader:
                # Ensure the entire batch is moved to the device
                batch = batch.to(device)
                optimizer.zero_grad()

                outputs = model(batch)
                loss = criterion(outputs, batch.y.view(-1, 1))  

                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            
            # Print loss every 10 epochs
            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")

        # Evaluate on the left-out test sample
        model.eval()
        with torch.no_grad():
            prediction = model(test_data).cpu().numpy()[0][0]  # Get prediction and move back to CPU

        # Store the true value and predicted value
        y_true.append(test_data.y.cpu().numpy()[0])
        y_pred.append(prediction)

    # Calculate RMSE and Pearson correlation coefficient (R-value)
    test_rmse = mean_squared_error(y_true, y_pred, squared=False)
    corr = pearsonr(y_true, y_pred)

    # Print the metrics
    print(f"\nModel: {model_class.__name__} | Test RMSE: {test_rmse:.4f} | Test R value (correlation): {corr[0]:.4f}")

    # Save predictions and metrics using the existing function
    model_name = model_class.__name__
    save_predictions_and_metrics(model_name, y_true, y_pred, corr[0], test_rmse)



features_file = '../Processed Data/rest_jhu_first_visit/'
labels_file = '../Processed Data/101_participants_40_regions_target_variable.csv'
prediction_label = 'Aphasia quotient'

X, y = load_data_3d(features_file, labels_file, prediction_label)

# Train the GNN model
train_and_evaluate_gnn(GCN, X, y)
#train_and_evaluate_3d(NeuralNetMLP, X, y)


Using device: cuda
Epoch [10/200], Loss: 4410.0552
Epoch [20/200], Loss: 4492.1182
Epoch [30/200], Loss: 4334.7894
Epoch [40/200], Loss: 4101.3366
Epoch [50/200], Loss: 3777.4244
Epoch [60/200], Loss: 2397.9063
Epoch [70/200], Loss: 2024.5803
Epoch [80/200], Loss: 1335.1375
Epoch [90/200], Loss: 821.1103
Epoch [100/200], Loss: 635.0001
Epoch [110/200], Loss: 669.1812
Epoch [120/200], Loss: 601.0932
Epoch [130/200], Loss: 688.0505
Epoch [140/200], Loss: 714.9279
Epoch [150/200], Loss: 604.3989
Epoch [160/200], Loss: 711.4239
Epoch [170/200], Loss: 551.0873
Epoch [180/200], Loss: 676.0183
Epoch [190/200], Loss: 683.0985
Epoch [200/200], Loss: 536.3320
Epoch [10/200], Loss: 4333.7787
Epoch [20/200], Loss: 4569.3975
Epoch [30/200], Loss: 4519.8047
Epoch [40/200], Loss: 3677.5983
Epoch [50/200], Loss: 2978.6760
Epoch [60/200], Loss: 2181.7789
Epoch [70/200], Loss: 1538.6178
Epoch [80/200], Loss: 832.6339
Epoch [90/200], Loss: 604.3848
Epoch [100/200], Loss: 688.4220
Epoch [110/200], Loss: 5

C:\Users\shark\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [49]:
import torch
print("CUDA Available: ", torch.cuda.is_available())
print("Number of GPUs: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")



CUDA Available:  True
Number of GPUs:  1
GPU Name:  NVIDIA GeForce RTX 4060 Laptop GPU


In [50]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from PIL import Image

def merge_metrics(directory):
    metrics_list = []

    # Loop through the files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('metrics.csv'):
            # Read each metrics CSV file
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)

            # Add a column for the model name (extracted from the filename)
            model_name = filename.replace('_metrics.csv', '')

            # Extract correlation and RMSE values
            r_corr = df.loc[df['Metric'] == 'Correlation', 'Value'].values[0]
            rmse = df.loc[df['Metric'] == 'RMSE', 'Value'].values[0]

            # Append the model name, correlation, and RMSE to the list
            metrics_list.append({'Model': model_name, 'R corr': r_corr, 'RMSE': rmse})

    # Convert the list into a DataFrame
    all_metrics = pd.DataFrame(metrics_list)

    # Save the merged metrics to a new CSV file
    output_file = os.path.join(directory, 'merged_metric.csv')
    all_metrics.to_csv(output_file, index=False)

    print(f"Merged metrics saved to {output_file}")
    return output_file

def plot_predictions(directory):
    # Loop through the files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('predictions.csv'):
            # Read each predictions CSV file
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)

            # Extract true and predicted values
            true_values = df['True Label'].values
            predicted_values = df['Predicted Label'].values

            # Calculate RMSE and Pearson correlation coefficient (R-value)
            rmse = mean_squared_error(true_values, predicted_values, squared=False)
            r_corr, _ = pearsonr(true_values, predicted_values)

            # Extract model name from the filename
            model_name = filename.replace('_predictions.csv', '')

            # Create the plot
            plt.figure(figsize=(8, 6))
            plt.scatter(true_values, predicted_values, label=f'R = {r_corr:.4f}\nRMSE = {rmse:.4f}', alpha=0.6)
            plt.plot([min(true_values), max(true_values)], [min(true_values), max(true_values)], color='red', linestyle='--')  # Diagonal line

            # Set title and labels
            plt.title(f'{model_name}')
            plt.xlabel('True')
            plt.ylabel('Predicted')
            
            # Show the legend with R and RMSE
            plt.legend(loc='lower right')

            # Show the plot
            plt.grid(True)            
            # Save the plot as a PNG file
            plot_filename = f"{model_name}_plot.png"
            plot_path = os.path.join(directory, plot_filename)
            plt.savefig(plot_path)
            plt.close()

            print(f"Plot saved to {plot_path}")

def combine_plots_to_pdf(directory):
    """
    Searches the specified directory for PNG files and combines them into a single PDF file.
    
    Args:
        directory (str): The path to the directory containing the PNG plot files.
    """
    # Initialize a list to store paths of PNG files
    image_paths = []

    # Search for PNG files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.png'):
            image_path = os.path.join(directory, filename)
            image_paths.append(image_path)

    # Combine the images into a single PDF
    if image_paths:
        images = []
        for image_path in image_paths:
            img = Image.open(image_path)

            # Convert image to 'RGB' mode if not already
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            images.append(img)

        # Save the first image and append the rest into a PDF
        pdf_path = os.path.join(directory, 'all_plots.pdf')
        images[0].save(pdf_path, save_all=True, append_images=images[1:])
        print(f"All plots combined and saved to {pdf_path}")
    else:
        print("No PNG plot files found.")

# Main workflow
if __name__ == '__main__':
    directory_path = '../Results' 

    # Merge the metrics and save to CSV
    merge_metrics(directory_path)

    # Plot all prediction files and save them as PNG
    plot_predictions(directory_path)

    # Combine all the PNG plots into a single PDF
    combine_plots_to_pdf(directory_path)

Merged metrics saved to ../Results\merged_metric.csv
Plot saved to ../Results\CNN_2d_plot.png
Plot saved to ../Results\CNN_plot.png
Plot saved to ../Results\DeepNeuralNetMLP_plot.png


C:\Users\shark\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\shark\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\shark\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\shark\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarnin

Plot saved to ../Results\GCN_plot.png
Plot saved to ../Results\Linear_Regression_plot.png
Plot saved to ../Results\NeuralNetwork_MLP_plot.png
Plot saved to ../Results\Random Forest_plot.png


C:\Users\shark\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\shark\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\shark\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\shark\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarnin

Plot saved to ../Results\SVR_plot.png
All plots combined and saved to ../Results\all_plots.pdf
